# IA prediction d'humeur BERT

instalation des transdormers

In [1]:
!pip install torch                
!pip install transformers 
!pip install pandas 
!pip install scikit-learn



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import json
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report





C:\Users\flocon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Charger les données


In [ ]:
import pandas as pd
import json

# Chemin relatif vers le fichier
file_path = "../data/yelp_academic_reviews4students.jsonl"

# Méthode 1 : Avec pandas (recommandé)
try:
    data = pd.read_json(file_path, lines=True)
    print(f"✓ Données chargées : {len(data)} lignes")
    print(f"✓ Colonnes : {list(data.columns)}")
    print(data.head())
except FileNotFoundError:
    print(f"✗ Fichier introuvable : {file_path}")
    print("Vérifiez que le fichier existe dans le dossier 'data' du répertoire parent")
except Exception as e:
    print(f"✗ Erreur : {e}")


✓ Données chargées : 1000000 lignes
✓ Colonnes : ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
                review_id                 user_id             business_id  \
0  J5Q1gH4ACCj6CtQG7Yom7g  56gL9KEJNHiSDUoyjk2o3Q  8yR12PNSMo6FBYx1u5KPlw   
1  HlXP79ecTquSVXmjM10QxQ  bAt9OUFX9ZRgGLCXG22UmA  pBNucviUkNsiqhJv5IFpjg   
2  JBBULrjyGx6vHto2osk_CQ  NRHPcLq2vGWqgqwVugSgnQ  8sf9kv6O4GgEb0j1o22N1g   
3  U9-43s8YUl6GWBFCpxUGEw  PAxc0qpqt5c2kA0rjDFFAg  XwepyB7KjJ-XGJf0vKc6Vg   
4  8T8EGa_4Cj12M6w8vRgUsQ  BqPR1Dp5Rb_QYs9_fz9RiA  prm5wvpp0OHJBlrvTj9uOg   

   stars  useful  funny  cool  \
0      2       1      0     0   
1      5       0      0     0   
2      5       0      0     0   
3      4       0      0     0   
4      5       0      0     0   

                                                text                date  
0  Went for lunch and found that my burger was me... 2018-04-04 21:09:53  
1  I needed a new tires for my wife's car. They

In [12]:
import re
import pandas as pd

def load_yelp_dataset(path):
    """
    Charge le dataset Yelp depuis un fichier JSONL
    """
    # Charger directement avec pandas (beaucoup plus simple et rapide)
    data = pd.read_json(path, lines=True)
    
    # Ne garder que les colonnes nécessaires
    df = pd.DataFrame({
        "texte": data["text"],
        "stars": data["stars"]
    })
    
    print(f"✓ Données chargées : {len(df)} lignes")
    print(f"✓ Distribution des étoiles :")
    print(df["stars"].value_counts().sort_index())
    
    return df


In [13]:
df = load_yelp_dataset("../data/yelp_academic_reviews4students.jsonl")
print(len(df))
print(df.head())


✓ Données chargées : 1000000 lignes
✓ Distribution des étoiles :
stars
1    153057
2     77630
3     98714
4    207953
5    462646
Name: count, dtype: int64
1000000
                                               texte  stars
0  Went for lunch and found that my burger was me...      2
1  I needed a new tires for my wife's car. They h...      5
2  Jim Woltman who works at Goleta Honda is 5 sta...      5
3  Been here a few times to get some shrimp.  The...      4
4  This is one fantastic place to eat whether you...      5


In [14]:
df = df.sample(200000, random_state=42)


In [15]:
def stars_to_label(stars):
    if stars <= 2:
        return 0   # negatif
    elif stars == 3:
        return 1   # neutre
    else:
        return 2   # positif

df["label"] = df["stars"].apply(stars_to_label)


# 2. Train / Test split


In [16]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["texte"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# 3. Tokenizer


In [17]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3
)

def tokenize(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

C:\Users\flocon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\flocon\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|███████


# 4. Dataset PyTorch


In [20]:
class MoodDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = MoodDataset(train_encodings, train_labels)
test_dataset = MoodDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# 5. Modèle


In [21]:
model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base",
    num_labels=3
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)


C:\Users\flocon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\flocon\.cache\huggingface\hub\models--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████

# 6. Entraînement


In [35]:
import torch
import sys

print("="*60)
print("🔍 DIAGNOSTIC COMPLET")
print("="*60)

# 1. Version de PyTorch
print(f"PyTorch version: {torch.__version__}")
print(f"Python version: {sys.version}")

# 2. CUDA disponible ?
print(f"\nCUDA disponible: {torch.cuda.is_available()}")
print(f"CUDA version compilée avec PyTorch: {torch.version.cuda if torch.version.cuda else 'Aucune'}")

# 3. Si CUDA non disponible, pourquoi ?
if not torch.cuda.is_available():
    print("\n⚠️ CUDA n'est pas disponible. Raisons possibles:")
    print("   1. Vous n'avez pas de GPU NVIDIA")
    print("   2. PyTorch CPU-only est installé")
    print("   3. Les drivers NVIDIA ne sont pas installés")
    
# 4. Device actuel
try:
    print(f"\n📍 Device utilisé: {device}")
except:
    print("\n📍 Variable 'device' non définie")

print("="*60)

🔍 DIAGNOSTIC COMPLET
PyTorch version: 2.10.0+cpu
Python version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]

CUDA disponible: False
CUDA version compilée avec PyTorch: Aucune

⚠️ CUDA n'est pas disponible. Raisons possibles:
   1. Vous n'avez pas de GPU NVIDIA
   2. PyTorch CPU-only est installé
   3. Les drivers NVIDIA ne sont pas installés

📍 Device utilisé: cpu


In [26]:
from tqdm import tqdm

epochs = 3
model.train()

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    total_loss = 0
    
    # Ajouter tqdm ici
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", unit="batch")
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        # Afficher la loss en temps réel
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    print("Loss moyenne :", total_loss / len(train_loader))


Epoch 1/3


Epoch 1:   3%|▎         | 519/20000 [15:07<9:27:52,  1.75s/batch, loss=0.9821] 


KeyboardInterrupt: 

In [22]:
epochs = 3

model.train()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Loss moyenne :", total_loss / len(train_loader))


Epoch 1/3


KeyboardInterrupt: 


# 7. Évaluation


In [ ]:

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\nRésultats :")
print(classification_report(
    true_labels,
    predictions,
    target_names=["negatif", "neutre", "positif"]
))


# 8. Test manuel


In [ ]:

def predict_mood(text):
    encoding = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    outputs = model(**encoding)
    prediction = torch.argmax(outputs.logits, dim=1).item()

    reverse_map = {0: "negatif", 1: "neutre", 2: "positif"}
    return reverse_map[prediction]

print("\nTest manuel :")
print(predict_mood("Je suis fatigué mais ça va"))
print(predict_mood("Cette journée est incroyable"))
print(predict_mood("Le train arrive à 8h"))
